In [2]:
import pandas as pd
import numpy as np

print("Pandas Version: " + pd.__version__)
print("Numpy Version: " + np.__version__)

Pandas Version: 1.2.4
Numpy Version: 1.18.5


In [3]:
trips = pd.read_csv("../../data/clean/trips.csv", parse_dates=["date_from", "date_until"])
trips["time_since_last_checkout"] = pd.to_timedelta(trips["time_since_last_checkout"])
trips.head(3)

,bike_id,user_id,date_from,date_until,start_station_name,start_station_id,end_station_name,end_station_id,booked_via,duration_in_min,distance_in_km,speed_in_kmh,time_since_last_checkout
0,119911,341973A96CDE0DF9792F6C844622735AE4216EBF,2014-01-01 00:02:51,2014-01-01 00:20:04,Enckeplatz / Hütten,131887,Königstraße / Struenseestraße,131650,Terminal HH_8 (-2624-),18,2.364129,7.880431,NaT
1,118994,665D79F269FA03F84FC61F3A7F7B078D7392EC0E,2014-01-01 00:07:45,2014-01-01 00:10:48,Isestraße / Hoheluftbrücke,140804,Isestraße / Hoheluftbrücke,140804,Terminal HH_63 (-2241-),4,0.000000,0.000000,NaT
2,143660,B46B52FDC494E46849DB84BF84F0B99C78358E59,2014-01-01 00:09:55,2014-01-01 00:26:20,Schulterblatt/Eifflerstraße,131648,Schulterblatt/Eifflerstraße,131648,Android SRH,17,0.000000,0.000000,NaT


In [4]:
user_groups = trips.groupby(["user_id"])

In [5]:
last_moment = trips.date_until.max()

In [6]:
user_trip_dates = trips.groupby(["user_id"], as_index=False).agg({"date_from": [np.min, np.max]})
user_trip_dates.columns = pd.Index(["user_id", "date_from_min", "date_from_max"])
user_trip_dates.set_index("user_id", inplace=True)

In [7]:
trips_count = user_groups.size().to_frame()
trips_count.rename(columns = {0: "trips_count"}, inplace=True)

In [8]:
active_period_in_days = user_groups.apply(
    lambda x: (x.date_from.max() - x.date_from.min()).days
).to_frame()
active_period_in_days.rename(columns = {0: "active_period_in_days"}, inplace=True)

In [9]:
days_since_first_activity = user_groups.apply(
    lambda x: (last_moment - x.date_from.min()).days
).to_frame()
days_since_first_activity.rename(columns = {0: "days_since_first_activity"}, inplace=True)

In [10]:
days_since_last_activity = user_groups.apply(
    lambda x: (last_moment - x.date_from.max()).days
).to_frame()
days_since_last_activity.rename(columns = {0: "days_since_last_activity"}, inplace=True)

In [11]:
# exclude all <= 0 (round trips)
mean_duration_in_min = trips[["user_id", "duration_in_min"]][trips.duration_in_min > 0].groupby("user_id").mean()
mean_duration_in_min.rename(columns = {"duration_in_min": "mean_duration_in_min"}, inplace=True)

In [12]:
# exclude all <= 0 (round trips)
mean_distance_in_km = trips[["user_id", "distance_in_km"]][trips.distance_in_km > 0].groupby("user_id").mean()
mean_distance_in_km.rename(columns = {"distance_in_km": "mean_distance_in_km"}, inplace=True)

In [13]:
# exclude all <= 0 (round trips)
mean_speed_in_kmh = trips[["user_id", "speed_in_kmh"]][trips.speed_in_kmh > 0].groupby("user_id").mean()
mean_speed_in_kmh.rename(columns = {"speed_in_kmh": "mean_speed_in_kmh"}, inplace=True)

In [14]:
users = pd.merge(user_trip_dates, trips_count, left_index=True, right_index=True)
users = pd.merge(users, active_period_in_days, left_index=True, right_index=True)
users = pd.merge(users, days_since_first_activity, left_index=True, right_index=True)
users = pd.merge(users, days_since_last_activity, left_index=True, right_index=True)
users = pd.merge(users, mean_duration_in_min, left_index=True, right_index=True)
users = pd.merge(users, mean_distance_in_km, left_index=True, right_index=True)
users = pd.merge(users, mean_speed_in_kmh, left_index=True, right_index=True)

In [15]:
# Calculate the mean time between trips in hours

mean_time_between_trips = trips[["user_id", "time_since_last_checkout"]].dropna()
mean_time_between_trips = mean_time_between_trips.groupby("user_id").mean(numeric_only=False)

users = pd.merge(
    users, mean_time_between_trips, left_index=True, right_index=True
).rename(columns={"time_since_last_checkout": "mean_time_between_trips"})

In [16]:
users.sample(5)

,date_from_min,date_from_max,trips_count,active_period_in_days,days_since_first_activity,days_since_last_activity,mean_duration_in_min,mean_distance_in_km,mean_speed_in_kmh,mean_time_between_trips
user_id,,,,,,,,,,
CC7E7BC9F4523DA36C043D08F31B3D1675B43E9B,2014-02-09 13:30:24,2014-06-04 07:20:30,4,114,1191,1077,15.000000,1.821289,7.292602,38 days 05:56:42
8A4806DB07106AD0497A006DFB2AEAAD685A51EF,2014-08-25 08:44:14,2015-10-17 16:48:08,19,418,994,576,13.526316,1.890442,8.485406,23 days 05:46:53
E2A3934D78F2E7CEB8FA214FC7301F99A7B85405,2015-08-15 16:56:46,2015-08-15 16:57:16,2,0,639,639,12.500000,1.542343,7.415109,0 days 00:00:30
0F8292BAEC6FB87EB9DE32BFA13B89EA2F49FEDE,2014-01-02 15:56:32,2015-07-11 15:07:53,130,554,1229,674,7.023077,1.004520,8.782799,4 days 07:14:58.302325581
0888FD1DC9AE697854B0B8665224F94D19A3FDF5,2014-05-20 00:27:05,2017-05-14 20:32:56,70,1090,1092,1,17.471429,2.549015,8.576590,15 days 19:25:18.130434782


In [17]:
users.to_csv("../../data/clean/users.csv", index=True)